In [2]:
%matplotlib inline

import functions as fun
from collections import Counter
import ast
import pickle
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence



from scipy.sparse import csr_matrix

import pydot
import os
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
from scipy.integrate import odeint 
import pandas as pd
import scipy as sp 
import string
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
import sklearn.model_selection as ms
from tensorflow import keras
import tensorflow as tf
import tensorboard as ts
from keras.preprocessing.text import Tokenizer
#tf.keras.backend.clear_session()

Using TensorFlow backend.






## Training Data

In [3]:
h = ['claimID', 'claim', 'label', 'claimURL', 'reason', 'categories', 'speaker', 'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']
df_train = pd.read_csv('../data/train.tsv',sep='\t',names=h,header=None)
df_dev = pd.read_csv('../data/dev.tsv',sep='\t',names=h,header=None)

## Snippet

In [4]:
h = ['snip_id', 'snip_1', 'snip_2','snip_url']
df_snip =  pd.DataFrame(columns=h)


dir = '..\data\snippets'
for filename in os.listdir(dir):
    df_row = pd.read_csv(os.path.join(dir, filename),names=h,sep='\t',header=None,engine='python',encoding='utf8' ,quoting=3)
    df_row.insert(loc = 0, column = 'claimID', value = filename )
    
    df_snip = df_snip.append(df_row)

df_snip.set_index(['claimID','snip_id'],inplace = False).head()    

snip_1  \
claimID    snip_id                                                      
abbc-00001 2        Record numbers march down Brisbane city to pro...   
           3        Fact check: Will Queensland's proposed abortio...   
           4        Queensland Council for Civil Liberties - Wikip...   
           5        Abortion law change expected from Queensland L...   
           6                                             The Struggle   

                                                               snip_2  \
claimID    snip_id                                                      
abbc-00001 2        Sep 4, 2018 ... THOUSANDS of people joined the...   
           3        Oct 3, 2018 ... Protestors have marched throug...   
           4        The Queensland Council for Civil Liberties (QC...   
           5        Jun 28, 2018 ... The Queensland Law Reform Com...   
           6        enforcing the State‟s controversial anti-march...   

                                                             snip_url  
claimID    snip_id                                                     
abbc-00001 2        http://catholicleader.com.au/news/record-numbe...  
           3        http://abortion-news.info/fact-check-will-quee...  
           4        https://en.wikipedia.org/wiki/Queensland_Counc...  
           5        http://www.abc.net.au/news/2018-06-29/abortion...  
           6        https://www.childrenbychoice.org.au/images/dow...

## Remove rows with missing snippets + clean

In [6]:
df_snip.reset_index(inplace=True)
df_train.reset_index(inplace=True)

In [7]:

df_train = df_train.loc[df_train['claimID'].isin(df_snip['claimID'].unique())]
df_dev =df_dev.loc[df_dev['claimID'].isin(df_snip['claimID'].unique())]


df_train.replace(to_replace = 'None',value=np.nan ,inplace = True)
df_train.replace(to_replace = "['None']",value=np.nan ,inplace = True)

df_dev.replace(to_replace = 'None',value=np.nan ,inplace = True)
df_dev.replace(to_replace = "['None']",value=np.nan ,inplace = True)

df_snip['snip_url'] = df_snip.apply(lambda x: fun.clean_url(x.snip_url),axis=1)

print('train shape:',np.shape(df_train))
print('dev shape:',np.shape(df_dev))
print('Snip shape:',np.shape(df_snip),'Unique:',len(df_snip['claimID'].unique()))


train shape: (24004, 13)
dev shape: (2988, 12)
Snip shape: (274657, 6) Unique: 30100


## Save data to pickle  

In [8]:

df_train.set_index('claimID',inplace = True)
df_snip.set_index(['claimID','snip_id'],inplace = True)
df_dev.set_index('claimID',inplace = True)

df_snip.to_pickle("./df_snip.pkl")
df_train.to_pickle("./df_dev.pkl")
df_train.to_pickle("./df_train.pkl")

In [ ]:
#df_snip = pd.read_pickle("./df_snip.pkl")
#df_train = pd.read_pickle("./df_train.pkl")

#df_train.reset_index(drop=False, inplace=True)
#df_snip.reset_index(drop=False, inplace=True)

In [ ]:
y_train_num =  df_train['label'].astype("category").cat.codes.values

with open('y_train_num','wb') as i:
       pickle.dump(y_train_num, i) 

## Metadata set

In [ ]:
df_meta = df_train[['claimURL','speaker','checker']].copy() 


df_meta['claimURL'] = df_meta.apply(lambda x: fun.clean_url(x.claimURL),axis=1)

df_meta['claimURL'] = df_meta['claimURL'].astype("category").cat.codes.values

df_meta['speaker'] = df_meta['speaker'].astype("category").cat.codes.values
df_meta['checker'] = df_meta['checker'].astype("category").cat.codes.values

    
df_meta.head() 

X_metadata = df_meta.to_numpy()

bin_enteties = fun.create_binary(df_train['entities'],30)

tags = df_train.apply(lambda x: str(x.tags).replace('[','').replace(']','').replace("'",'').split(',')   ,axis=1)

bin_tags = fun.create_binary(tags,30)



X_metadata = np.append(X_metadata, bin_enteties, axis=1)

with open('X_meta','wb') as i:
       pickle.dump(X_metadata, i)


df_meta_num = pd.DataFrame(data = np.hstack((df_train.index.to_numpy().reshape(-1,1),X_metadata))     )        
        
with open('df_meta_num','wb') as i:
       pickle.dump(df_meta_num, i)        



## Claim set

In [ ]:
df_claim = df_train['claim'].str.lower().copy()

words_list = [i if type(i)==str else '' for i in df_claim]

t = Tokenizer()
t.fit_on_texts(words_list)

sequences = t.texts_to_sequences(words_list)

X_claim = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=50)


with open('X_claim','wb') as i:
       pickle.dump(X_claim, i)
            

            
df_claim_num = pd.DataFrame(data = np.hstack((df_train.index.to_numpy().reshape(-1,1),X_claim))   )        
        
with open('df_claim_num','wb') as i:
       pickle.dump(df_claim_num, i) 

## Snippet set

In [ ]:
df_snip['snip_url'] = df_snip.apply(lambda x: fun.clean_url(x.snip_url),axis=1)

df_snip['snip_1'] = df_snip['snip_1'].str.lower()
df_snip['snip_2'] = df_snip['snip_2'].str.lower()

df_snip['snip_url'] = df_snip['snip_url'].astype("category").cat.codes.values


df_snip.reset_index(drop=False, inplace=True)


In [ ]:

words_list1 = [i if type(i)==str else '' for i in df_snip['snip_1']]
sequences1 = t.texts_to_sequences(words_list1)
snip1 = tf.keras.preprocessing.sequence.pad_sequences(sequences1, maxlen=20)

words_list2 = [i if type(i)==str else '' for i in df_snip['snip_2']]
sequences2 = t.texts_to_sequences(words_list2)
snip2 = tf.keras.preprocessing.sequence.pad_sequences(sequences2, maxlen=80)


In [ ]:

snip_info = df_snip[['claimID','snip_id','snip_url']].to_numpy()
snip = np.append(snip_info, np.append(snip1,snip2,axis=1), axis=1)


df_snip_num =pd.DataFrame(data=snip)
df_snip_num

with open('df_snip_num','wb') as i:
       pickle.dump(df_snip_num, i) 
        
        

In [ ]:
df_snip_num.head()

## Flat Snippet set (very very slow)

In [ ]:

snippet_list = []

for q,i in enumerate(df_train.index):
    pass
    row = []
    for i,r in df_snip_num[ df_snip_num[0]==i].iterrows():
       
        row.append(r[1:].to_list())
    row = [snip for snippets in row for snip in snippets]
    
    snippet_list.append(row)
    
   

In [ ]:
X_snip_flat =  tf.keras.preprocessing.sequence.pad_sequences(snippet_list, maxlen=1005)

with open('X_snip_flat','wb') as i:
       pickle.dump(X_snip_flat, i) 
        

## jj

In [ ]:
X_claim